In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# Define the directory containing the objects images
objects_dir = '/kaggle/input/dl-dataset/DL Dataset'

# Define the list of target objects classes
classes = ['beds', 'bikes', 'cars', 'cats', 'chair', 'cycle', 'dogs', 'flowers', 'horses', 'person']

accuracy_values = []
loss_values = []
# Define the transformation to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom dataset class for loading the objects images
class ObjectDataset(Dataset):
    def __init__(self, objects_dir, classes, transform=None):
        self.objects_dir = objects_dir
        self.classes = classes
        self.transform = transform
        self.image_files = []
        self.labels = []

        # Initialize label encoder
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(classes)

        # Iterate over each class folder
        for classe in classes:
            classe_dir = os.path.join(objects_dir, classe)
            if os.path.isdir(classe_dir):
                files = os.listdir(classe_dir)
                self.image_files.extend(files)
                self.labels.extend([classe] * len(files))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.objects_dir, self.labels[idx], self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        label = self.label_encoder.transform([label])[0]

        return image, label


# Create the dataset
dataset = ObjectDataset(objects_dir, classes, transform=transform)

# Split the dataset into training and testing sets
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Create the dataloaders for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Modify the last fully connected layer for the number of classes
num_classes = len(classes)

# Define the custom VGG16-like model
class CustomVGG16(nn.Module):
    def __init__(self, num_classes):
        super(CustomVGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Create an instance of the custom VGG16-like model
model = CustomVGG16(num_classes)


# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Track the loss
        running_loss += loss.item() * images.size(0)

        # Track the accuracy
        _, predicted_labels = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted_labels == labels).sum().item()

    epoch_loss = running_loss / len(dataset)
    epoch_accuracy = (correct_predictions / total_predictions) * 100

    accuracy_values.append(epoch_accuracy)
    loss_values.append(epoch_loss)


    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

plt.plot(range(1, num_epochs+1), accuracy_values)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.show()


plt.plot(range(1, num_epochs+1), loss_values)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.show()


# Save the trained model
torch.save(model.state_dict(), '/kaggle/input/dl-model-path/DL/model.pth')



import numpy as np
model.eval()

# Load and preprocess the input image
input_image_path = '/kaggle/input/testing-dataset/testing/chair_test.jpg'  # Replace with the path to your input image
input_image = Image.open(input_image_path).convert('RGB')
input_image_transformed = transform(input_image)
input_image_tensor = input_image_transformed.unsqueeze(0)  # Add an extra dimension for batch size

# Perform classification
with torch.no_grad():
    output = model(input_image_tensor)
    predicted_labels = torch.argmax(output, dim=1)
    predicted_class = classes[predicted_labels.item()]
    confidence = torch.softmax(output, dim=1)[0, predicted_labels].item() * 100

# Print the predicted class and confidence
print(f"Predicted Class: {predicted_class}")
print(f"Confidence: {confidence:.2f}%")

# Display the testing picture with the predicted class
plt.imshow(np.transpose(input_image_transformed, (1, 2, 0)))
plt.axis('off')
plt.title(f'Testing Picture\nPredicted Class: {predicted_class}', fontsize=12)
plt.show()
